In [1]:
# 기본적인 Naive RAG -> 조건부엣지 검사 -> 웹 검색 -> 쿼리 재작성
import os
from dotenv import load_dotenv
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver



load_dotenv()

True

In [2]:
# 랭그래프에서는 Retriever 와 Chain 을 따로 생성해서 다른 노드에 위치시킨다
from rag.pdf import PDFRetrievalChain

pdf = PDFRetrievalChain(["SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

In [3]:
search_result = pdf_retriever.invoke("엔스로픽에 투자한 기업과 투자금액을 알려주세요.")
search_result

[Document(id='3e4c95a5-d6b8-4cb9-b213-3a372b9829b8', metadata={'source': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 13, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}, page_content='1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화\nKEY Contents\nn 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은\n구글과 클라우드 서비스 사용 계약도 체결\nn 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인\n앤스로픽 및 오픈AI와 협력을 확대하는 추세\n£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공'),
 Document(id='50fc6ba3-9966-459c-bd59-4c40d2b2659e', metadata={'source': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 13, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "

In [4]:
answer = pdf_chain.invoke(
    {
        "question":"엔스로픽에 투자한 기업과 투자금액을 알려주세요.",
        "context":search_result,
        "chat_history":[],
    }
)
print(answer)

- 구글(Google): 최대 20억 달러 투자 합의; 우선 5억 달러 투자, 향후 15억 달러 추가 투자 예정; 2023년 2월에 이미 5억 5,000만 달러 투자.
- 아마존(Amazon): 최대 40억 달러의 투자 계획 발표(2023년 9월).

Source
- SPRI_AI_Brief_2023년12월호_F.pdf (page 13)


In [5]:
# State 정의
from typing import Annotated, TypedDict, List
from langgraph.graph.message import add_messages

# 노드에서 다른 노드로 전달할 때 알려줄 State (상태결과)
class MyState(TypedDict):
    question: Annotated[str, "Question"]        # 질문
    context: Annotated[str, "Context"]          # 문서 검색 결과
    answer: Annotated[str, "Answer"]            # 답변
    messages: Annotated[list, add_messages]         # 메시지 누적 list

In [6]:
# Node 정의 (리트리브 노드, LLM 노드)
from langchain_teddynote.messages import messages_to_history
from rag.utils import format_docs

# 문서 검색 노드
def retrieve_document(state : MyState) -> MyState:
    # 마지막 질문
    latest_question = state["question"]
    # 문서에서 유사도 검색
    retrieved_docs = pdf_retriever.invoke(latest_question)
    retrieved_docs = format_docs(retrieved_docs)
    return MyState(context=retrieved_docs)

# LLM 답변 노드
def llm_answer(state : MyState)->MyState:
    # 질문을 상태에서 가져온다
    latest_question = state["question"]
    # 검색된 문서를 상태에서 가져옵니다
    context = state["context"]
    #체인을 호출하여 답변을 생성한다
    response = pdf_chain.invoke({
        "question" : latest_question,
        "context": context,
        "chat_history":messages_to_history(state["messages"])
    })
    return MyState(answer=response, messages=[("user", latest_question), ("assistant", response)])


In [7]:
# 상태그래프에 Node 추가 -> Edge로 연결 -> 상태그래프 컴파일
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# 상태그래프 생성
workflow = StateGraph(MyState)

# 노드 추가
workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)

# 엣지로 연결
workflow.add_edge("retrieve", "llm_answer")
workflow.add_edge("llm_answer", END)
workflow.set_entry_point("retrieve")

# 컴파일
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

EDGE = C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe


[INFO] Starting Chromium download.


Browser OK: Edg/139.0.3405.125


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


In [9]:
# 그래프 실행
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid

config = RunnableConfig(recursion_limit=20, configurable={"thread_id":random_uuid()})

inputs = MyState(question="앤스로릭에 투자한 기업과 투자금액을 알려주세요")

invoke_graph(app, inputs, config)


🔄 Node: retrieve 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
context:
<document><content>1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공</content><source>SPRI_AI_Brief_2023년12월호_F.pdf</source><page>14</page></document>
<document><content>4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를</content><source>SPRI_AI_Brief_2023년12월호_F.pdf</source><page>14</page></document>
<document><content>£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,

In [10]:
stream_graph(app, inputs, config)


🔄 Node: llm_answer 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
- 구글: 앤스로픽에 최대 20억 달러 투자 합의; 우선 투자 5억 달러, 2023년 2월에 이미 5억 5,000만 달러 투자
- 아마존: 앤스로픽에 최대 40억 달러 투자 계획 공개(2023년 9월)

Source
- SPRI_AI_Brief_2023년12월호_F.pdf (page 14)

In [11]:
outputs = app.get_state(config).values

print(f'Question: {outputs["question"]}')
print("===" * 20)
print(f'Answer:\n{outputs["answer"]}')

Question: 앤스로릭에 투자한 기업과 투자금액을 알려주세요
Answer:
- 구글: 앤스로픽에 최대 20억 달러 투자 합의; 우선 투자 5억 달러, 2023년 2월에 이미 5억 5,000만 달러 투자
- 아마존: 앤스로픽에 최대 40억 달러 투자 계획 공개(2023년 9월)

Source
- SPRI_AI_Brief_2023년12월호_F.pdf (page 14)
